In [ ]:
from requests_html import AsyncHTMLSession
import asyncio
import json
import numpy as np
import pandas as pd
import time


In [58]:
tours = []

for year in [2021, 2022, 2023, 2024]:
    with open(f'data/wtt/tournaments{year}.json') as f:
        tours.extend(json.load(f)[0]['rows'])

In [59]:
tours = [tour for tour in tours if tour['EventType'].startswith('WTT') or tour['EventType'] == 'Continental Championships' or tour['EventType'].startswith('ITTF') or tour['EventType'] == 'Olympic Games' or tour['EventType'] == 'World Championships']
tours = [tour for tour in tours if 'Youth' not in tour['EventType'] and 'Exhibition' not in tour['EventType']]

In [60]:
tf = pd.DataFrame(tours)
tf.drop_duplicates(subset=['EventId'], keep='last', inplace=True)
tf

,PageLink,EventName,EventType,EventTypeId,Country,City,ContinentCode,Subcontinent,StartDateTime,EndDateTime,...,EventCode,Comments,EventDateChangeId,EventId,FromStartDate,FromEndDate,ToStartDate,ToEndDate,ShowInCalendar,Type
0,None,2021 ITTF Pan American Championships,Continental Championships,79,Peru,Lima,americas,south america,2021-11-13T00:00:00,2021-11-19T00:00:00,...,0,None,NaN,2234,None,None,None,None,None,None
3,None,Tokyo 2020 Olympic Games,Olympic Games,38,Japan,Tokyo,asia,east asia,2021-07-23T00:00:00,2021-08-08T00:00:00,...,0,None,NaN,2345,None,None,None,None,None,None
5,None,2021 World Table Tennis Championships Finals,WTTC,37,USA,Houston,americas,northern america,2021-11-23T00:00:00,2021-11-29T00:00:00,...,0,None,NaN,2346,None,None,None,None,None,None
6,None,WTT Contender Doha 2021,WTT Contender,34,Qatar,Doha,asia,west asia,2021-02-28T11:30:00,2021-03-06T23:59:59,...,0,None,NaN,2410,None,None,None,None,None,None
7,None,WTT Star Contender Doha 2021,WTT Star Contender,35,Qatar,Doha,asia,west asia,2021-03-05T19:30:00,2021-03-13T23:59:00,...,0,None,NaN,2411,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,None,ITTF Africa Olympic Qualification event,ITTF International Open,80,Rwanda,Kigali,africa,east africa,2024-05-16T00:00:00,2024-05-18T00:00:00,...,None,None,NaN,2946,None,None,None,None,None,None
164,None,WTT Finals Fukuoka 2024,WTT Finals,75,Japan,Kitakyushu,asia,east asia,2024-11-20T00:00:00,2024-11-24T00:00:00,...,None,None,NaN,2947,None,None,None,None,None,None
165,None,2024 South American Championships,WTTC Finals Regional Qualifications,83,Paraguay,Asunción,americas,south america,2024-06-10T00:00:00,2024-06-16T00:00:00,...,None,None,NaN,2960,None,None,None,None,None,None
166,None,WTT Feeder Prishtina 2024,WTT Feeder,81,Kosovo,Prishtina,europe,southern europe,2024-10-28T00:00:00,2024-11-01T00:00:00,...,None,None,NaN,2964,None,None,None,None,None,None


In [61]:
tf.drop(inplace=True, columns=['PageLink', 'FromStartDate', 'ToStartDate', 'FromEndDate', 'ToEndDate', 'ShowInCalendar', 'Type', 'EventCode', 'Comments', 'EventDateChangeId', 'Tags'])


In [62]:
rows = tf[
    tf.EventName.str.contains('U11', case=False)
    | tf.EventName.str.contains('U13', case=False)
    | tf.EventName.str.contains('U15', case=False)
    | tf.EventName.str.contains('U17', case=False)
    | tf.EventName.str.contains('U19', case=False)
    | tf.EventName.str.contains('Senior', case=False)
    | tf.EventName.str.contains('Youth', case=False)
    | tf.EventName.str.contains('Junior', case=False)
]
rows

,EventName,EventType,EventTypeId,Country,City,ContinentCode,Subcontinent,StartDateTime,EndDateTime,EventId
22,ITTF World Youth Championships,ITTF International Open,80,Portugal,Vila Nova De Gaia,europe,southern europe,2021-12-02T00:00:00,2021-12-08T00:00:00,2511
44,2022 Caribbean Senior Championships,Continental Championships,79,Cuba,Havana,americas,caribbean,2022-03-03T00:00:00,2022-03-08T00:00:00,2565
57,2022 South East Asian U15&U19 Training Camp,Continental Championships,79,Thailand,Bangkok,asia,south east asia,2022-06-15T00:00:00,2022-06-16T00:00:00,2596
58,26th Southeast Asian Junior & Cadet Table Tenn...,Continental Championships,79,Thailand,Bangkok,asia,south east asia,2022-06-17T00:00:00,2022-06-22T00:00:00,2597
60,26th Asian Junior and Cadet Table Tennis Champ...,Continental Championships,79,Bahrain,Manama,asia,west asia,2022-06-23T00:00:00,2022-06-26T00:00:00,2599
66,2022 ITTF-Africa Senior Championships,Continental Championships,79,Algeria,None,africa,north africa,2022-09-03T00:00:00,2022-09-09T00:00:00,2609
68,ITTF-Oceania Senior Championships,Continental Championships,79,Australia,Melbourne,oceania,australasia,2022-09-05T00:00:00,2022-09-10T00:00:00,2615
93,2023 ITTF-Oceania Senior and Youth Championships,Continental Championships,79,Australia,,oceania,australasia,2023-09-04T00:00:00,2023-09-09T00:00:00,2718
111,2023 South American U11 & U13 Championships,ITTF International Open,80,Paraguay,Asunción,americas,south america,2023-05-09T00:00:00,2023-05-14T00:00:00,2756
116,2023 ITTF-Africa Senior Championships,Continental Championships,79,Tunisia,Rades,africa,north africa,2023-09-11T00:00:00,2023-09-17T00:00:00,2787


In [20]:
tf.drop(tf.index[tf.EventId==2474], inplace=True)

In [14]:
tf.to_csv('data/tournaments_wtt.tsv', sep='\t', index=False)

In [54]:
tf = pd.read_csv('data/tournaments_wtt.tsv', sep='\t')

In [66]:
res = {}

In [117]:
for curr in tf[~tf.EventId.isin(res)].itertuples():
    s = AsyncHTMLSession()
    print(curr)
    h = await s.get(f'https://worldtabletennis.com/eventInfo?selectedTab=Results&eventId={curr.EventId}')
    await h.html.arender(timeout=60, keep_page=True)
    # await h.html.page.waitForSelector(".generic_btn")
    # await asyncio.sleep(1)
    lis = await s.get(f'https://wttapigateway-new.azure-api.net/prod/api/cms/GetOfficialResult?EventId={curr.EventId}&DocumentCode=TTE')
    data = lis.json()
    await s.close()
    print(data)
    if 'statusCode' not in data:
        res[curr.EventId] = data

Pandas(Index=6, EventName='ITTF-ATTU Asian Table Tennis Championships', EventType='Continental Championships', EventTypeId=79, Country='Qatar', City='Doha ', ContinentCode='asia', Subcontinent='west asia', StartDateTime='2021-09-28T00:00:00', EndDateTime='2021-10-05T00:00:00', EventId=2474)
{'statusCode': 401, 'message': 'Not authorized'}
Pandas(Index=29, EventName='2022 ITTF World Team Championships Finals Presented by China Telecom', EventType='WTTC', EventTypeId=37, Country='China', City='Chengdu', ContinentCode='asia', Subcontinent='east asia', StartDateTime='2022-09-30T00:00:00', EndDateTime='2022-10-09T00:00:00', EventId=2535)
{'statusCode': 401, 'message': 'Not authorized'}


In [134]:
res2 = []
for v in res.values():
    for m in v:
        m['eventId'] = int(m['eventId'])
        res2.append(m)

In [18]:
mf = pd.DataFrame(res2)
mf.to_csv('data/tournaments/res.tsv', sep='\t', index=False)

In [32]:
mf = pd.read_csv('data/tournaments/res.tsv', sep='\t')
mf

,id,eventId,documentCode,subEventType
0,403899,2234,TTEMTEAM--------------FNL-00010000--------,Men's Team
1,402897,2234,TTEWTEAM--------------FNL-00010000--------,Women's Team
2,402565,2234,TTEMTEAM--------------SFNL00020000--------,Men's Team
3,402561,2234,TTEMTEAM--------------SFNL00010000--------,Men's Team
4,400249,2234,TTEWTEAM--------------SFNL00010000--------,Women's Team
...,...,...,...,...
21212,2060216,2535,TTEMTEAM--------------GP0100040000--------,Men's Team
21213,2060189,2535,TTEMTEAM--------------GP0600010000--------,Men's Team
21214,2059941,2535,TTEMTEAM--------------GP0100030000--------,Men's Team
21215,2059506,2535,TTEWTEAM--------------GP0300010000--------,Women's Team
